# Note:
#### This notebook is Assigment 2, praticing sparkSQL commands based on another program called Covid19_demo2.ipynb file.  Practice that demo program first and then fill in the Spark Commands for the questions that begin with **#Q** (e..g, #Q1, #Q2, #Q3, etc)  and **submit the runned ipynb file. There are a total of 24 questions. **

#### This program reads two dataset files called "**A.csv**" and "**B.csv**". You should store them in G-drive. You can stoe them in any folder of your choice or use the same folder hierarchy as shown inside this program. 

#### Since the file names used in this assignment is A.csv and B.csv, I used a new data frame name as in A1_DF, A2_DF, A3_DF, B1_DF, B2_DF, etc.

#### If you have any questions on Spark, Colb, or this Assignment 2, contact my TA. If you see any ambiguity, please contact me or my TA.

#### You have 2 weeks to complete this assignment.

# Run cells below to setup spark environment

In [ ]:
!pip install pyspark

### Mounting this notebook with google drive "/contesnt/gdrive/" is the path to google drive




In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Import necessary libararies


*   SparkSession is used to create spark dataframe
*   import functions as f then when you want to use function you just simply call it by "f" instead of typing the full word "functions"
* functions module contain many builtin function such as concatenate, add_date etc.
* UserDefinedFucntion is a module that allow you to create your on function  that will be applied to each entry of dataframe



In [ ]:
from pyspark.sql import functions as f
from pyspark.sql.functions import UserDefinedFunction
from datetime import date, datetime, timedelta
from pyspark.sql.types import *

### Creating SparkSession object
Spark session objects can be created by using SparkSession.builder.getorCreated(). You also have to specify APP_NAME for your sparkSession that you want to work on.


In [ ]:
from pyspark.sql import SparkSession
APP_NAME = "preprocessing"
spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
spark

### Creating a Dataframe from csv files
You can read csv files from google drive by providing the path “/gdrive/My Drive/”. This path is the path to access files in your google drive. Since we’ve put dataset in “data” folder, the path name should be “/gdrive/My Drive/data/filename.csv”

You can create a spark dataframe by calling sparkSession object name “spark” that you have created in the previous step. 
spark.read(‘csv’): specify that you want spark to read .csv file.
option(‘header’,’true’): contain header to the dataframe
option(‘inferSchema’, ‘true’): ensure that we detect the correct type for each column. Without this option, the default type will be string for all columns.
 printSchema to see the Schema of data


In [ ]:
#Create DF called train
#file_folder = './gdrive/My Drive/data/'
file_folder = "./gdrive/My Drive/Covid19/data/"
A_DF = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(file_folder + 'A.csv')
A_DF.show()
# null values in csv file is marked as "null" without quotes.

+---+--------------+--------------+----+----+----------+--------------+----------+
| Id|Province/State|Country/Region| Lat|Long|      Date|ConfirmedCases|Fatalities|
+---+--------------+--------------+----+----+----------+--------------+----------+
|  1|          null|   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|
|  2|          null|   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|
|  3|          null|   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|
|  4|          null|   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|
|  5|          null|   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|
|  6|          null|   Afghanistan|33.0|65.0|2020-01-27|           0.0|       0.0|
|  7|          null|   Afghanistan|33.0|65.0|2020-01-28|           0.0|       0.0|
|  8|          null|   Afghanistan|33.0|65.0|2020-01-29|           0.0|       0.0|
|  9|          null|   Afghanistan|33.0|65.0|2020-01-30|           0.0|       0.0|
| 10

### Rename Columns
We can rename columns of dataframe by directly calling command withColumnRenamed() from a data frame object. In this case “train” is a dataFrame object.
* withColumnRenamed(): to rename columns result in a new data frame because the dataframe consists of Row RDDs. They are immutable
* We have to assign it back to “train” (train = train.withColumnRenamed()) in order to modify the dataframe
* In order to see content in spark dataframe, the command .show() is called directly from a dataframe object
* .show() command will show the first 20 rows of the dataframe

In [ ]:
# rename columns
A2_DF = A_DF.withColumnRenamed('Province/State','Province_State').withColumnRenamed('Country/Region','Country_Region')
A2_DF.show()

+---+--------------+--------------+----+----+----------+--------------+----------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|
+---+--------------+--------------+----+----+----------+--------------+----------+
|  1|          null|   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|
|  2|          null|   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|
|  3|          null|   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|
|  4|          null|   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|
|  5|          null|   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|
|  6|          null|   Afghanistan|33.0|65.0|2020-01-27|           0.0|       0.0|
|  7|          null|   Afghanistan|33.0|65.0|2020-01-28|           0.0|       0.0|
|  8|          null|   Afghanistan|33.0|65.0|2020-01-29|           0.0|       0.0|
|  9|          null|   Afghanistan|33.0|65.0|2020-01-30|           0.0|       0.0|
| 10

In [ ]:
# crete two dataframes (ynames, cpd) that store some column names which will be used later
ynames = ['ConfirmedCases', 'Fatalities']
ny = len(ynames)
ny

2

In [ ]:
cp = ['Country_Region','Province_State']
cpd = cp + ['Date']
cpd

['Country_Region', 'Province_State', 'Date']

In [ ]:
#Q1 crete a new variable called geography that stores columns 'Lat' and 'Long', which will be used later and then run.

geography = ['Lat', 'Long']
geography

['Lat', 'Long']

### Fill missing values with empty string
An example of filling missing value with empty string in "Provinve_State" column

In [ ]:
# filling missing values with empty string
A3_DF = A2_DF.fillna({'Province_State' : '' })
A3_DF.show()

+---+--------------+--------------+----+----+----------+--------------+----------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|
+---+--------------+--------------+----+----+----------+--------------+----------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|
|  3|              |   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|
|  4|              |   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|
|  5|              |   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|
|  6|              |   Afghanistan|33.0|65.0|2020-01-27|           0.0|       0.0|
|  7|              |   Afghanistan|33.0|65.0|2020-01-28|           0.0|       0.0|
|  8|              |   Afghanistan|33.0|65.0|2020-01-29|           0.0|       0.0|
|  9|              |   Afghanistan|33.0|65.0|2020-01-30|           0.0|       0.0|
| 10

### Create new column from existing columns
* withColumn() can also create a new column as well
* by specify column name, if the column name doesn’t exist in the dataframe, it will automatically create a new column with the name
* In this case “key” is the column name. 
* after that, build in function f.concat() is call to concat value in two columns with “_”
build in function f.lit() is necessary. It is use to pass a constant or value to operate with value in a dataframe i.e. in this case, “_’ is passed into f.lit() fiction in order to let it connect between two fields value
* For example, for a tuple “province_state = Busan”, “Country_regoin = South Korea” the result of will be “Busan_South Korea”


In [ ]:
# Example of creating a new column "PSCR" by concatinating two columns connected by "_"
#lit() PySpark SQL functions lit() and typedLit() are used to add a new column to DataFrame by assigning a literal or constant value. Both these functions return Column type as return type.
A4_DF = A3_DF.withColumn('PSCR',f.concat(f.col('Province_State'),f.lit('_'),f.col('Country_Region')))
A4_DF.show()

+---+--------------+--------------+----+----+----------+--------------+----------+------------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|        PSCR|
+---+--------------+--------------+----+----+----------+--------------+----------+------------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|_Afghanistan|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|_Afghanistan|
|  3|              |   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|_Afghanistan|
|  4|              |   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|_Afghanistan|
|  5|              |   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|_Afghanistan|
|  6|              |   Afghanistan|33.0|65.0|2020-01-27|           0.0|       0.0|_Afghanistan|
|  7|              |   Afghanistan|33.0|65.0|2020-01-28|           0.0|       0.0|_Afghanistan|
|  8|              |   Afghanistan|33.0|

Repeat the same procedure for the dataset A (A.csv) to dataset B (B.csv)
rename columns
filling missing value empty string
modify existing column
f.concat() and f.lit()

In [ ]:
# In Q2, Q3, and Q4, you are basically repeating the same steps as you did for dataset A.csv file
#Q2 Read the dataset B.csv using the similar steps.
file_folder = "./gdrive/My Drive/Covid19/data/"
B_DF = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(file_folder + 'B.csv')
B_DF.show()
# rename columns as above 
#Q3 #Rename columns 'Province/State' to 'Province_State' by calling command withColumnRenamed() from a dataframe
B2_DF = B_DF.withColumnRenamed('Province/State','Province_State').withColumnRenamed('Country/Region','Country_Region')
B2_DF.show()


#Q4 filling missing (null) values for 'Province_State' with empty string as done in the A.csv.
B3_DF = B2_DF.fillna({'Province_State' : '' })
B3_DF.show()




#Q5 create a new column "PSRC" by concatinating two columns as done in dataset A.csv
#You need to combine the Province_state and the country_region columns.
B4_DF = B3_DF.withColumn('PSCR',f.concat(f.col('Province_State'),f.lit('_'),f.col('Country_Region')))
B4_DF.show()



+----------+--------------+--------------+----+----+----------+
|ForecastId|Province/State|Country/Region| Lat|Long|      Date|
+----------+--------------+--------------+----+----+----------+
|         1|          null|   Afghanistan|33.0|65.0|2020-03-12|
|         2|          null|   Afghanistan|33.0|65.0|2020-03-13|
|         3|          null|   Afghanistan|33.0|65.0|2020-03-14|
|         4|          null|   Afghanistan|33.0|65.0|2020-03-15|
|         5|          null|   Afghanistan|33.0|65.0|2020-03-16|
|         6|          null|   Afghanistan|33.0|65.0|2020-03-17|
|         7|          null|   Afghanistan|33.0|65.0|2020-03-18|
|         8|          null|   Afghanistan|33.0|65.0|2020-03-19|
|         9|          null|   Afghanistan|33.0|65.0|2020-03-20|
|        10|          null|   Afghanistan|33.0|65.0|2020-03-21|
|        11|          null|   Afghanistan|33.0|65.0|2020-03-22|
|        12|          null|   Afghanistan|33.0|65.0|2020-03-23|
|        13|          null|   Afghanista

### Aggreagte funtion (max)

In [ ]:
# tmax is the last day of training; Either syntax below is working
tmax = A4_DF.agg (f.max(A4_DF.Date)).show()
#tmax = train.agg({'Date':'max'}).show()

+----------+
| max(Date)|
+----------+
|2020-03-24|
+----------+



Collect (Action) - Return all the elements of the dataset as an array at the driver program. This is usually useful after a filter or other operation that returns a sufficiently small subset of the data.

#collect()[0]: Row(max(Date)='2020-03-24')

#collect(): [Row(max(Date)='2020-03-24')]

In [ ]:
tmax = A4_DF.agg({'Date':'max'}).collect()[0]['max(Date)']
tmax

'2020-03-24'

Convert the string to a datetime object
print() 




In [ ]:
print(type(tmax))
tmax = datetime.strptime(tmax, '%Y-%m-%d').date()
print(tmax,type(tmax))

<class 'str'>
2020-03-24 <class 'datetime.date'>


The same example on the test data

In [ ]:
# fmax is the last day of B dataset
#Q6 Find the last day from the B dataset using the max function.
fmax = B4_DF.agg (f.max(B4_DF.Date)).show()

+----------+
| max(Date)|
+----------+
|2020-04-23|
+----------+



In [ ]:
#Q7 Convert the above last day dataframe into a date string
fmax = B4_DF.agg({'Date':'max'}).collect()[0]['max(Date)']
fmax

'2020-04-23'

In [ ]:
#Q8 Convert the date string o a datetime object
print(type(fmax))
fmax = datetime.strptime(fmax, '%Y-%m-%d').date()
print(fmax,type(fmax))

<class 'str'>
2020-04-23 <class 'datetime.date'>


An example of using min aggregate function and collect the value

type(tmin) below is a string that is selected from the result of collect()[0]['min(Date)']


In [ ]:
tmin = A4_DF.agg({'Date':'min'}).collect()[0]['min(Date)']
fmin = B4_DF.agg({'Date':'min'}).collect()[0]['min(Date)']
#tmin,fmin
print(tmin)
print(fmin)

2020-01-22
2020-03-12


In [ ]:
#Q9 Convert both of the above computed strings to the datetime object
tmin = datetime.strptime(tmin, '%Y-%m-%d').date()
fmin = datetime.strptime(fmin, '%Y-%m-%d').date()
print(tmin,type(tmin))
print(fmin,type(fmin))

2020-01-22 <class 'datetime.date'>
2020-03-12 <class 'datetime.date'>


### Using SQL statement to exact column and merge dataframes
We can use SQL command in spark SQL by 
* first using command creatieOrReplaceTempView(“viewname”) directly from a dataframe that you want to use SQL statement
* In this case, we create view name “train” from train dataframe
* view name “test” from test data frame
* We use command spark.sql(“SQL statement)
* The result is a new dataframe
* From example below first SQL statement is used to create a new dataframe from ‘test
* “SELECT Country_Region, Province_State, ForecastId, Date FROM test” the new dataframe from this query is assigned to “test_sub_col”
* then view name “sub_test” is created from test_sub_col dataframe
* then the last SQL statement is used to merge two view together e.g. “train” and “sub_test”
* the result is assigned to merge_train 


In [ ]:
# In order to execute a SQL command, you have to create a view of the dataframe first using createOrReplaceTempView()
# and use it inside spark.sql command
A4_DF.createOrReplaceTempView('A4_V')
A5_DF = spark.sql('select * from A4_V')
A5_DF.show(3)
A5_DF.count()
B4_DF.createOrReplaceTempView('B4_V')
B5_DF = spark.sql('select * from B4_V')
B5_DF.show(3)
B5_DF.count()

B_sub_col_DF = spark.sql('SELECT Country_Region, Province_State, ForecastId, Date FROM B4_V')
B_sub_col_DF.show(3)
B_sub_col_DF.count()

+---+--------------+--------------+----+----+----------+--------------+----------+------------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|        PSCR|
+---+--------------+--------------+----+----+----------+--------------+----------+------------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|_Afghanistan|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|_Afghanistan|
|  3|              |   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|_Afghanistan|
+---+--------------+--------------+----+----+----------+--------------+----------+------------+
only showing top 3 rows

+----------+--------------+--------------+----+----+----------+------------+
|ForecastId|Province_State|Country_Region| Lat|Long|      Date|        PSCR|
+----------+--------------+--------------+----+----+----------+------------+
|         1|              |   Afghanistan|33.0|65.0|2020-03-12|_Afghanis

12212

In [ ]:
#Q10. Use SQL SELECT command to store Country_Region, Date, ConfirmedCases, and Fatalities. Call the new dataframe "CDCF_DF". 
CDCF_DF = spark.sql('SELECT Country_Region, Date, ConfirmedCases, Fatalities FROM A4_V')
#Q11 Show(20)
CDCF_DF.show(20)
#Q12, count the number of rows of "CDCF_DF" dataframe.
CDCF_DF.count()

+--------------+----------+--------------+----------+
|Country_Region|      Date|ConfirmedCases|Fatalities|
+--------------+----------+--------------+----------+
|   Afghanistan|2020-01-22|           0.0|       0.0|
|   Afghanistan|2020-01-23|           0.0|       0.0|
|   Afghanistan|2020-01-24|           0.0|       0.0|
|   Afghanistan|2020-01-25|           0.0|       0.0|
|   Afghanistan|2020-01-26|           0.0|       0.0|
|   Afghanistan|2020-01-27|           0.0|       0.0|
|   Afghanistan|2020-01-28|           0.0|       0.0|
|   Afghanistan|2020-01-29|           0.0|       0.0|
|   Afghanistan|2020-01-30|           0.0|       0.0|
|   Afghanistan|2020-01-31|           0.0|       0.0|
|   Afghanistan|2020-02-01|           0.0|       0.0|
|   Afghanistan|2020-02-02|           0.0|       0.0|
|   Afghanistan|2020-02-03|           0.0|       0.0|
|   Afghanistan|2020-02-04|           0.0|       0.0|
|   Afghanistan|2020-02-05|           0.0|       0.0|
|   Afghanistan|2020-02-06| 

17892

In [ ]:
#Q13 Perform groupby() by columns "Country_Region" and "Date" and then show(20)
CDCF_DF.groupby('Country_Region', 'Date').count().show(20)

+--------------+----------+-----+
|Country_Region|      Date|count|
+--------------+----------+-----+
|       Albania|2020-02-05|    1|
|       Albania|2020-02-18|    1|
|       Algeria|2020-03-19|    1|
|       Andorra|2020-02-14|    1|
|         Aruba|2020-01-30|    1|
|        Canada|2020-01-22|   11|
|         Chile|2020-03-01|    1|
|       Croatia|2020-01-26|    1|
|       Croatia|2020-02-02|    1|
|       Denmark|2020-03-01|    2|
|       Finland|2020-02-03|    1|
|        France|2020-02-09|    8|
|        France|2020-03-19|    8|
|         Gabon|2020-03-12|    1|
|     Guatemala|2020-03-22|    1|
|      Guernsey|2020-01-26|    1|
|      Guernsey|2020-03-16|    1|
|      Honduras|2020-01-23|    1|
|          Iraq|2020-03-17|    1|
|         Italy|2020-02-21|    1|
+--------------+----------+-----+
only showing top 20 rows



In [ ]:
# We want to use unionByName() operation between A4 and B5. 
#But in order to execute it, both dataframes must have the same number of columns and the same column names.
# So, check their schema and remove some column names and change a column name.
#Q14 show the schema of A4 dataframe
A4_DF.printSchema()
#Q15 Show the schema of B5 dataframe
B5_DF.printSchema()
#Q16 from A4 frame, drop two columns "ConfirmedCases" and "Fatalitie" and then show(5). Use a single command to drop both columns.
A5_DF = A4_DF.drop('ConfirmedCases').drop('Fatalities')
A5_DF.show(5)
#Q17 from B5, rename column called "ForecastID" into "Id" and then show(5).
B6_DF = B5_DF.withColumnRenamed('ForecastId','Id')
B6_DF.show(5)

root
 |-- Id: integer (nullable = true)
 |-- Province_State: string (nullable = false)
 |-- Country_Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- ConfirmedCases: double (nullable = true)
 |-- Fatalities: double (nullable = true)
 |-- PSCR: string (nullable = true)

root
 |-- ForecastId: integer (nullable = true)
 |-- Province_State: string (nullable = false)
 |-- Country_Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- PSCR: string (nullable = true)

+---+--------------+--------------+----+----+----------+------------+
| Id|Province_State|Country_Region| Lat|Long|      Date|        PSCR|
+---+--------------+--------------+----+----+----------+------------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|_Afghanistan|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|_Afghanistan|


In [ ]:
#Q18 Perform unionByName() operation between A4 and B5 and call the result "AB_unioned" dataframe.
AB_unioned = A5_DF.unionByName(B6_DF)
#Q19 show(20) of "AB_unioned"
AB_unioned.show(20)
#Q20 count the number of rows of "AB_unioned"
AB_unioned.count()

+---+--------------+--------------+----+----+----------+------------+
| Id|Province_State|Country_Region| Lat|Long|      Date|        PSCR|
+---+--------------+--------------+----+----+----------+------------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|_Afghanistan|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|_Afghanistan|
|  3|              |   Afghanistan|33.0|65.0|2020-01-24|_Afghanistan|
|  4|              |   Afghanistan|33.0|65.0|2020-01-25|_Afghanistan|
|  5|              |   Afghanistan|33.0|65.0|2020-01-26|_Afghanistan|
|  6|              |   Afghanistan|33.0|65.0|2020-01-27|_Afghanistan|
|  7|              |   Afghanistan|33.0|65.0|2020-01-28|_Afghanistan|
|  8|              |   Afghanistan|33.0|65.0|2020-01-29|_Afghanistan|
|  9|              |   Afghanistan|33.0|65.0|2020-01-30|_Afghanistan|
| 10|              |   Afghanistan|33.0|65.0|2020-01-31|_Afghanistan|
| 11|              |   Afghanistan|33.0|65.0|2020-02-01|_Afghanistan|
| 12|              |

30104

In [ ]:
#Q21 Perform left outer join between A4 and B5 and call it "AB_joined and select only the columns from the A4 dataframe"
B5_DF.createOrReplaceTempView('B5_V')
AB_joined = spark.sql('SELECT * FROM A4_V LEFT OUTER JOIN B5_V' )
#Q22 Show (20) of "AB_joined"
AB_joined.show(20)
#Q23 Count #rows of "AB_joined"
AB_joined.count()

+---+--------------+--------------+----+----+----------+--------------+----------+------------+----------+--------------+--------------+----+----+----------+------------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|        PSCR|ForecastId|Province_State|Country_Region| Lat|Long|      Date|        PSCR|
+---+--------------+--------------+----+----+----------+--------------+----------+------------+----------+--------------+--------------+----+----+----------+------------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|_Afghanistan|         1|              |   Afghanistan|33.0|65.0|2020-03-12|_Afghanistan|
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|_Afghanistan|         2|              |   Afghanistan|33.0|65.0|2020-03-13|_Afghanistan|
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|_Afghanistan|         3|              |   Afghanistan|33.0|65.0

218497104

In [ ]:
#Q24 Save "AB_unioned" into a csv file in your disk and then show all the file names in the same folder you just saved.
import os
AB_unioned.write.save(file_folder + 'AB_unioned.csv')
os.listdir(file_folder)


['B.csv', 'A.csv', 'AB_unioned.csv']